<a href="https://colab.research.google.com/github/arkeyan/Indicators/blob/main/MaestroCrypto_Cakma_Ustad_RSI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/rongardF/tvdatafeed pandas numpy matplotlib ta
!pip install tradingview-screener==2.5.0

import pandas as pd
import numpy as np
from tvDatafeed import TvDatafeed, Interval
from tradingview_screener import get_all_symbols
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

def calculate_rsi(df, period=66):
    delta = df['close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.ewm(span=period, adjust=False).mean()
    avg_loss = loss.ewm(span=period, adjust=False).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    return rsi

def detect_rsi_break(df):
    rsi = calculate_rsi(df)
    df['RSI'] = rsi

    pivots = {'low': [], 'high': []}

    for i in range(1, len(rsi)-1):
        if rsi.iloc[i-1] > rsi.iloc[i] < rsi.iloc[i+1]:
            pivots['low'].append((i, rsi.iloc[i]))
        if rsi.iloc[i-1] < rsi.iloc[i] > rsi.iloc[i+1]:
            pivots['high'].append((i, rsi.iloc[i]))

    if not pivots['low'] or not pivots['high']:
        return False, rsi.iloc[-1]

    for low in pivots['low']:
        for high in pivots['high']:
            if low[0] > high[0] and low[1] < high[1]:
                if rsi.iloc[-1] > high[1]:
                    return True, rsi.iloc[-1]

    return False, rsi.iloc[-1]

def get_interval(period):
    intervals = {
        '30min': Interval.in_30_minute,
        '45min': Interval.in_45_minute,
        '2h': Interval.in_2_hour,
        '4h': Interval.in_4_hour,
        '1day': Interval.in_daily
    }
    return intervals.get(period, Interval.in_daily)  # Varsayılan olarak günlük

def main():
    tv = TvDatafeed()

    # Kullanıcıdan dönem ve borsa seçimini al
    period = input("Lütfen dönem seçimini yapın (30min, 45min, 2h, 4h, 1day): ").strip().lower()
    interval = get_interval(period)

    exchange = input("Lütfen borsa seçimini yapın (BINANCE, MEXC): ").strip().upper()
    if exchange not in ['BINANCE', 'MEXC']:
        print("Geçersiz borsa seçimi. Lütfen 'BINANCE' veya 'MEXC' girin.")
        return

    try:
        # Kripto sembollerini çek
        cryptos = get_all_symbols(market='crypto')

        # USDT paritelerini filtrele
        usdt_pairs = [symbol for symbol in cryptos if symbol.endswith('USDT') and exchange in symbol] #filtrelemek istediğiniz bir uzantı varsa buradan yapılabilir...sadece tırnak içi değişecek.

        usdt_pairs = sorted(usdt_pairs)
    except Exception as e:
        print(f"Kripto paraları çekerken hata alındı: {e}")
        return

    for crypto in usdt_pairs:
        try:
            df = tv.get_hist(crypto, exchange, interval)
            if df is None or df.empty:
                continue
            detected, rsi_value = detect_rsi_break(df)
            if detected and rsi_value > 30:  # rsi 30 uzerındeki kırılımları alıyor.. bu değeride değiştirebilirsiniz...
                print(f"{crypto} için RSI düşeni kırdı! RSI Değeri: {rsi_value:.2f}")
        except Exception as e:
            print(f"{crypto} için veri işleme hatası: {e}")

if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("Program kesildi.")
#tarama ortalama 5 - 7 dk arası sürüyor... tamamlandıktan sonra rsi değeri büyükten küçüğe doğru tekrardan filtreleyecek period seçimi sonrası taramayı başlatıp bekleyeceksiniz.. kolay gelsin.

  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-l7gv4_ll
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-l7gv4_ll
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done
  Created wheel for tvdatafeed: filename=tvdatafeed-2.1.0-py3-none-any.whl size=17532 sha256=7fa0f20dc23cf8cbbcc0b467826f14dd9ab122c0413a93b58ad254a119924e54
  Stored in directory: /tmp/pip-ephem-wheel-cache-o8968tkj/wheels/e4/32/1e/21ebcacc6549d75fae3bf3ff75cee8fcbe4e5c189d88fbd5a1
Successfully built tvdatafeed
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.6 MB/s eta 0:00:00
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=750f69ff716f21aed53dbbd390dcaebe0d5e0eadbc09f94517ef101f558936fb
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a

Lütfen dönem seçimini yapın (30min, 45min, 2h, 4h, 1day): 1day
Lütfen borsa seçimini yapın (BINANCE, MEXC): BINANCE
BINANCE:ARDRUSDT için RSI düşeni kırdı! RSI Değeri: 30.49
BINANCE:ARKUSDT için RSI düşeni kırdı! RSI Değeri: 41.68
Program kesildi.
